In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.stats import linregress 
import numpy as np
import math
import matplotlib.ticker as ticker
from scipy.odr import ODR, Model, Data, RealData


In [2]:
# First method for finding the saturation value. Compares i with i+1 at a certain level in %
def find_saturation_value(column_data):
    for i in range(len(column_data) - 1):
        if abs(column_data.iloc[i] - column_data.iloc[i + 1]) <= 101 * abs(column_data.iloc[i]):
            saturation = column_data.iloc[i]
            break
        else:
            saturation = None
    return saturation


In [3]:
lowLevel_plot = True 
highLevel_plot = True
verbose = True
pedestal_plot = True
fit = True

In [4]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd

def plot_histograms(T: pd.DataFrame, output_pdf: str = "histograms.pdf", bins: int = 50):

    # Identify numeric columns
    numeric_cols = T.select_dtypes(include='number').columns

    if len(numeric_cols) == 0:
        raise ValueError("No numeric columns found in the DataFrame.")

    # Create and save histograms
    with PdfPages(output_pdf) as pdf:
        for col in numeric_cols:
            plt.figure(figsize=(8, 5))
            plt.hist(T[col].dropna(), bins=bins, color='steelblue', alpha=0.75, edgecolor='black')
            plt.title(f"Histogram of {col}")
            plt.xlabel(col)
            plt.ylabel("Frequency")
            plt.grid(True, linestyle='--', alpha=0.6)
            pdf.savefig()
            plt.close()

        # Metadata
        d = pdf.infodict()
        d['Title'] = 'Measurement Stability Histograms'
        d['Author'] = 'Monica Seglar-Arroyo'
        d['Subject'] = 'Stability analysis of measurement data'
        d['Keywords'] = 'histogram, stability, measurements'

    print(f"✅ Saved {len(numeric_cols)} histograms to '{output_pdf}'")


In [5]:
def Histograms(folder_path,dataset_number,output_path,output_path_lowlevel,datetest,boardname):

    outputpath = f"{output_path}"
    outputpath_lowlevel = f"{output_path_lowlevel}"
    
    filterOrder= ['FW5', 'FW5','FW5','FW5','FW5','FW5','FW5','FW5','FW5','FW5'] 
    sensorsID = ['0.2', '0.3','1.2', '1.3','2.2', '2.3', '3.2', '3.3', '4.2', '4.3']
    #sensorsID = ['0.0','0.1','1.0','1.1','2.0','2.1','3.0','3.1','4.0','4.1']
    wavelength = ['1064']
   
    if not os.path.exists(outputpath_lowlevel):
        os.makedirs(outputpath_lowlevel)
        os.makedirs(outputpath)
        print(f"Directory '{outputpath_lowlevel}' created.")

    if not os.path.exists(outputpath):
        os.makedirs(outputpath)
        print(f"Directory '{outputpath}' created.")

    for wl in wavelength:
        pedestalsADC = []
        pedestalsADC_stderr = []
        pedestalsRefPD = []
        pedestalsRefPD_stderr = []
        slopes = []
        intercepts= []
        rCoes = []
        slopes_stderr = []
        intercepts_stderr = []
        saturationADCs = []
        if wl== '532':
        # Rename DataFrame columns
            strg_L = 'Laser Current (mA)'
            #temp = '20'
        elif wl == '1064':
            strg_L = 'Laser Power (mW)'
            #temp = '25'

        for l, sensorID in enumerate(sensorsID):
            filter = filterOrder[l]
            pedestal_list = []
            main_list = []
            for number in np.arange(1,dataset_number+1):
                filename = f'{folder_path}/{datetest}_{boardname}_{sensorID}_{wl}_{filter}_{number}.txt'
                # if the file does not exist, skip to the next iteration

                # Read the first line as a separate DataFrame
                df = pd.read_csv(filename, sep="\t", header=None)
                
                # Split the first row into a separate DataFrame
                first_row = df.iloc[[0]]  # Select the first row as a DataFrame
                last_row = df.iloc[[-1]]
                pedestal_list.append(first_row)  # Add the first row to pedestal list
                pedestal_list.append(last_row)  # Add the first row to pedestal list

                # Get the remaining rows (from the second row onward)
                main_data = df.iloc[1:-1]  # Select all rows except the first and last
                main_list.append(main_data)  # Add to main data list

            # Concatenate the first rows (metadata or header info) and the main data
            TP = pd.concat(pedestal_list, ignore_index=True)
            T = pd.concat(main_list, ignore_index=True)
            TP.columns = ['Date-Hour', 'L', 'TotalSum', 'TotalSquareSum', 'meanRefPD', 'stdRefPD', 'Tem', 'RH', 'TotalCounts']
            T.columns = ['Date-Hour', 'L', 'TotalSum', 'TotalSquareSum', 'meanRefPD', 'stdRefPD', 'Tem', 'RH', 'TotalCounts']

            meanADC = T['TotalSum']/T['TotalCounts']
            stdADC = np.sqrt((T['TotalSquareSum']-T['TotalCounts']*meanADC**2)/(T['TotalCounts']-1))
            T['meanADC'] = meanADC
            T['stdADC'] = stdADC 

            pedestal_meanADC = TP['TotalSum']/TP['TotalCounts']
            TP['meanADC'] = pedestal_meanADC
            TP['stdADC'] = np.sqrt((TP['TotalSquareSum']-TP['TotalCounts']*pedestal_meanADC**2)/(TP['TotalCounts']-1))

            Tsys = T[['Tem', 'RH']].copy()
            #print(f'Starting the fitting procedure for calibration coefficient for sensorID {sensorID} for laser {wl} nm')
            plot_histograms(Tsys, output_pdf=f"{outputpath_lowlevel}/Histogram_Stability_Sensor{sensorID}_{wl}nm.pdf", bins=50)


In [6]:
Histograms('./Data/setup2/13112025_18R2_SideA/data',3,"./Data/setup2/13112025_18R2_SideA/plots","./Data/setup2/13112025_18R2_SideA/plots/LowLevel",'13112025','18R2')


✅ Saved 2 histograms to './Data/setup2/13112025_18R2_SideA/plots/LowLevel/Histogram_Stability_Sensor0.2_1064nm.pdf'
✅ Saved 2 histograms to './Data/setup2/13112025_18R2_SideA/plots/LowLevel/Histogram_Stability_Sensor0.3_1064nm.pdf'
✅ Saved 2 histograms to './Data/setup2/13112025_18R2_SideA/plots/LowLevel/Histogram_Stability_Sensor1.2_1064nm.pdf'
✅ Saved 2 histograms to './Data/setup2/13112025_18R2_SideA/plots/LowLevel/Histogram_Stability_Sensor1.3_1064nm.pdf'
✅ Saved 2 histograms to './Data/setup2/13112025_18R2_SideA/plots/LowLevel/Histogram_Stability_Sensor2.2_1064nm.pdf'
✅ Saved 2 histograms to './Data/setup2/13112025_18R2_SideA/plots/LowLevel/Histogram_Stability_Sensor2.3_1064nm.pdf'
✅ Saved 2 histograms to './Data/setup2/13112025_18R2_SideA/plots/LowLevel/Histogram_Stability_Sensor3.2_1064nm.pdf'
✅ Saved 2 histograms to './Data/setup2/13112025_18R2_SideA/plots/LowLevel/Histogram_Stability_Sensor3.3_1064nm.pdf'
✅ Saved 2 histograms to './Data/setup2/13112025_18R2_SideA/plots/LowLeve

In [7]:
def find_saturation_from_derivate(T):
        
        T.sort_values(by=['meanADC'])
        # Extract x and y values
        x = T['meanRefPD'].values
        y = T['meanADC'].values

        # Compute the first derivative using central differences
        dy_dx = np.gradient(y, x)

        # Add the derivative to the DataFrame
        T['dy_dx'] = dy_dx
        #print(T['dy_dx'])
        
        # Define a threshold for identifying the plateau
        #threshold = 10001
        threshold = 10

        # Identify indices where dy_dx is not flat
        #non_flat_indices_aux = np.where(np.abs(dy_dx) >= threshold)[0]

        # Get some extra distance to be sure that there is a reduced number of flat indices
        #non_flat_indices = non_flat_indices_aux[:-5]


        flat_indices = np.where(np.abs(dy_dx) < threshold)[0]
        if flat_indices.size > 0:
                # Find the index and shift it by two for safety
                first_flat_index = flat_indices[0] 
                satADC=y[first_flat_index]
                safe_index = first_flat_index -2
                T_non_flat = T.iloc[:safe_index]
        else:
                satADC=-1
                T_non_flat = T
        
        #print(satADC)
        # Extract the corresponding rows
        #T_non_flat = T.iloc[non_flat_indices]
        
        return T_non_flat, satADC

In [8]:
def ObtainMeanAndStd(mean_values,err_values):
    safe_stderr = np.where(err_values.values == 0, 1e-10, err_values.values)
    weights = 1 / np.array(safe_stderr)**2
    weighted_mean = np.sum(weights * mean_values) / np.sum(weights)
    weighted_mean_error = np.sqrt(1 / np.sum(weights))
    weighted_variance = np.sum(weights * (mean_values - weighted_mean)**2) / np.sum(weights)
    weighted_std_dev = np.sqrt(weighted_variance)
    return weighted_mean, weighted_std_dev


In [9]:
def ADC_analysis_std(folder_path,dataset_number,output_path,output_path_lowlevel,datetest,boardname):

    outputpath = f"{output_path}"
    outputpath_lowlevel = f"{output_path_lowlevel}"

    sensorsID = ['0.0','0.1','0.2','0.3','1.0','1.1','1.2','1.3','2.0','2.1','2.2','2.3','3.0','3.1','3.2','3.3','4.0','4.1','4.2','4.3']
    wavelength = ['1064','532']
   
    if not os.path.exists(outputpath_lowlevel):
        os.makedirs(outputpath_lowlevel)
        os.makedirs(outputpath)
        print(f"Directory '{outputpath_lowlevel}' created.")

    if not os.path.exists(outputpath):
        os.makedirs(outputpath)
        print(f"Directory '{outputpath}' created.")

    for wl in wavelength:
        pedestalsADC = []
        pedestalsADC_stderr = []
        pedestalsRefPD = []
        pedestalsRefPD_stderr = []
        slopes = []
        intercepts= []
        rCoes = []
        slopes_stderr = []
        intercepts_stderr = []
        saturationADCs = []
        if wl== '532':
        # Rename DataFrame columns
            strg_L = 'Laser Current (mA)'
            #temp = '20'
        elif wl == '1064':
            strg_L = 'Laser Power (mW)'
            #temp = '25'

        for l in sensorsID:
            sensorID=l
            pedestal_list = []
            main_list = []
            for number in np.arange(1,dataset_number+1):
                filename = f'{folder_path}/{datetest}_{boardname}_{sensorID}_{wl}_{number}.txt'
                # Read the first line as a separate DataFrame
                df = pd.read_csv(filename, sep="\t", header=None)
                
                # Split the first row into a separate DataFrame
                first_row = df.iloc[[0]]  # Select the first row as a DataFrame
                last_row = df.iloc[[-1]]
                pedestal_list.append(first_row)  # Add the first row to pedestal list
                pedestal_list.append(last_row)  # Add the first row to pedestal list

                # Get the remaining rows (from the second row onward)
                main_data = df.iloc[1:-1]  # Select all rows except the first and last
                main_list.append(main_data)  # Add to main data list

            # Concatenate the first rows (metadata or header info) and the main data
            TP = pd.concat(pedestal_list, ignore_index=True)
            T = pd.concat(main_list, ignore_index=True)
            TP.columns = ['Date-Hour', 'L', 'TotalSum', 'TotalSquareSum', 'meanRefPD', 'stdRefPD', 'Tem', 'RH', 'TotalCounts']
            T.columns = ['Date-Hour', 'L', 'TotalSum', 'TotalSquareSum', 'meanRefPD', 'stdRefPD', 'Tem', 'RH', 'TotalCounts']

            meanADC = T['TotalSum']/T['TotalCounts']
            stdADC = np.sqrt((T['TotalSquareSum']-T['TotalCounts']*meanADC**2)/(T['TotalCounts']-1))
            T['meanADC'] = meanADC
            T['stdADC'] = stdADC
            
            pedestal_meanADC = TP['TotalSum']/TP['TotalCounts']
            TP['meanADC'] = pedestal_meanADC
            TP['stdADC'] = np.sqrt((TP['TotalSquareSum']-TP['TotalCounts']*pedestal_meanADC**2)/(TP['TotalCounts']-1))

            # meanPM = meanADC*0.61e-3 #0.61mV/ADCcount  
            # stdPM = stdADC*0.61e-3
            # # T['meanPM']=meanPM
            # T['stdPM']=stdPM
            print(f'Starting the fitting procedure for calibration coefficient for sensorID {sensorID} for laser {wl} nm')
            #print(T)
            #print(TP)

            if fit:
                T, saturationADC = find_saturation_from_derivate(T)
                #saturationADC=-1
                res = linregress(T['meanRefPD'], T['meanADC'])
                
                slopes.append(res.slope)
                intercepts.append(res.intercept)
                rCoes.append(res.rvalue)
                slopes_stderr.append(res.stderr)
                intercepts_stderr.append(res.intercept_stderr)
                saturationADCs.append(saturationADC)
            
            pedestalsADC_wmean, pedestalsADC_wstderr = ObtainMeanAndStd(TP['meanADC'],TP['stdADC'])
            pedestalsPD_wmean, pedestalsPD_wstderr = ObtainMeanAndStd(TP['meanRefPD'],TP['stdRefPD'])


            pedestalsADC.append(pedestalsADC_wmean)
            #sqrt((∑X^2-N*meanADC^2/(N-1))
            pedestalsADC_stderr.append(pedestalsADC_wstderr) # This errors should be added in quadrature
            pedestalsRefPD.append(pedestalsPD_wmean) 
            pedestalsRefPD_stderr.append(pedestalsPD_wstderr)# This errors should be added in quadrature


            if(verbose and fit):
                print(f'Calibration coefficient found for sensorID {sensorID} for laser {wl} nm = {res.slope} +/- {res.stderr} in ADC/V')

            if(lowLevel_plot):
                fig = plt.figure(2)
                plt.errorbar(T['L'], T['meanADC'], yerr=T['stdADC'], fmt='.', markersize=10, linewidth=1)
                plt.ylabel('mean ADC counts')
                plt.xlabel(strg_L)
                plt.grid()
                plt.title(f'Plot sensorID {sensorID}, wavelength {wl} nm')
                plt.tight_layout()
                plt.savefig(f'{outputpath_lowlevel}/{datetest}_{boardname}_{sensorID}_{wl}_meanADC_L.png',dpi=199)  # Display the current figure
                plt.close(fig)

                fig = plt.figure(4)
                plt.errorbar(T['L'], T['meanRefPD'], yerr=T['stdRefPD'], fmt='.', markersize=10, linewidth=1)
                plt.ylabel('Mean ref PD (V)')
                plt.xlabel(strg_L)
                plt.grid()
                plt.title(f'Plot sensorID {sensorID}, wavelength {wl} nm')
                plt.tight_layout()
                plt.savefig(f'{outputpath_lowlevel}/{datetest}_{boardname}_{sensorID}_{wl}_meanRefPD_L.png',dpi=199)  # Display the current figure
                #plt.show()
                plt.close(fig)

                #the label here are incorrect since meanPM=meanADC*0.61e-3 #0.61mV/ADCcoun. Monica proposed from the baffle paper: 
                #Voltage/ADCcount =3.3V/2**10=0.00322 V/ADCcount
                #4.6 uW/ADCcount*1ADCcount/0.00322V = 1428 uW/V

                if fit:
                    fig = plt.figure(6)
                    plt.errorbar(T['meanRefPD'], T['meanADC'], yerr=T['stdADC'], fmt='.', markersize=10, linewidth=1)
                    plt.plot(T['meanRefPD'], res.intercept + res.slope*T['meanRefPD'], 'r', label='fitted line')
                    plt.ylabel('Mean ADC counts')
                    plt.xlabel('Mean ref PD (V)')
                    plt.grid()
                    plt.title(f'Plot sensorID {sensorID}, wavelength {wl} nm')
                    plt.tight_layout()
                    plt.savefig(f'{outputpath_lowlevel}/{datetest}_{boardname}_{sensorID}_{wl}_meanADC_meanRefPD.png',dpi=199)  # Display the current figure
                    #plt.show()
                    plt.close(fig)
            
        if(highLevel_plot and fit):
                tabdata = pd.DataFrame()
                tabdata["sensorID"] = sensorsID
                tabdata["slope"] = slopes
                tabdata["intercept"] = intercepts
                tabdata["rCoe"] = rCoes
                tabdata["slopes_stderr"] = slopes_stderr
                tabdata["intercept_stderr"] = intercepts_stderr
                tabdata["saturationADC"] = saturationADCs
                
                fig = plt.figure(300)
                plt.errorbar(range(len(intercepts)), intercepts,intercepts_stderr, fmt='.',  markersize=10, linewidth=1)
                plt.axhline(y=np.mean(intercepts), color='tab:olive', linestyle='-')
                plt.fill_between(range(-1,len(intercepts)+1), (np.mean(intercepts)-np.std(intercepts)), (np.mean(intercepts)+np.std(intercepts)), color='tab:olive', alpha=0.2)
                plt.title(f'Intercepts at {wl} nm. Value = {np.mean(intercepts):.1f} +- {np.std(intercepts):.1f}')
                plt.xlabel('SensorID')
                plt.ylabel('ADC counts')
                plt.xticks(np.arange(len(intercepts)), sensorsID, rotation=20)
                plt.xlim([-1,len(sensorsID)])
                plt.grid()
                plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_noshade_intercepts.png',dpi=199)  # Display the current figure
                plt.close()
                
                fig = plt.figure(400)
                plt.errorbar(range(len(rCoes)), rCoes, fmt='.', markersize=10,linewidth=1)
                plt.axhline(y=np.mean(rCoes), color='tab:olive', linestyle='-')
                plt.fill_between(range(-1,len(rCoes)+1), (np.mean(rCoes)-np.std(rCoes)), (np.mean(rCoes)+np.std(rCoes)), color='tab:olive', alpha=0.2)
                plt.title(f'rCoes at {wl} nm. Value = {np.mean(rCoes):.1f} +- {np.std(rCoes):.1f}')
                plt.xlabel('SensorID')
                plt.ylabel('')
                plt.xticks(np.arange(len(rCoes)), sensorsID, rotation=20)
                plt.xlim([-1,len(sensorsID)])
                plt.grid()
                plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_rcoes.png',dpi=199)  # Display the current figure
                plt.close()
                
                fig = plt.figure(500)
                plt.title(f'Saturation ADC at {wl} nm. Value = {np.mean(saturationADCs):.1f} +- {np.std(saturationADCs):.1f}')
                plt.plot(range(len(saturationADCs)), saturationADCs,'.')
                plt.axhline(y=np.mean(saturationADCs), color='tab:olive', linestyle='-')
                plt.fill_between(range(-1,len(saturationADCs)+1), (np.mean(saturationADCs)-np.std(saturationADCs)), (np.mean(saturationADCs)+np.std(saturationADCs)), color='tab:olive', alpha=0.2)
                plt.xlabel('SensorID')
                plt.ylabel('')
                plt.xticks(np.arange(len(saturationADCs)), sensorsID, rotation=20)
                plt.xlim([-1,len(sensorsID)])
                plt.grid()
                plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_saturationADC.png',dpi=199)  # Display the current figure
                plt.close()

                sensorsID1=sensorsID[:16]
                sensorsID2=sensorsID[-4:]
                slopes1=slopes[:16]
                slopes2=slopes[-4:]
                intercepts1=intercepts[:16]
                intercepts2=intercepts[-4:]
                rCoes1=rCoes[:16]
                rCoes2=rCoes[-4:]
                slopes_stderr1=slopes_stderr[:16]
                slopes_stderr2=slopes_stderr[-4:]
                intercepts_stderr1=intercepts_stderr[:16]
                intercepts_stderr2=intercepts_stderr[-4:]
                saturationADCs1=saturationADCs[:16]
                saturationADCs2=saturationADCs[-4:]
                
                #Uncertainty of Weighted Mean: Precision based on individual measurement uncertainties.
                #Intervariability: Variability among the measurements (random error, i.e. embedded systematic uncertainties).
                #Purely Systematic Error: Accounts for known or estimated biases in the measurements (temperature/humidity????)

                weights1 = 1 / np.array(slopes_stderr1)**2
                weighted_mean1 = np.sum(weights1 * slopes1) / np.sum(weights1)
                weighted_mean_error1 = np.sqrt(1 / np.sum(weights1))
                weighted_variance1 = np.sum(weights1 * (slopes1 - weighted_mean1)**2) / np.sum(weights1)
                weighted_std_dev1 = np.sqrt(weighted_variance1)
                intervariability1 = np.std(slopes1, ddof=1)  # Standard deviation (sample)
                combined_sigma1 = np.sqrt(weighted_std_dev1**2+intervariability1**2)
                
                weights2 = 1 / np.array(slopes_stderr2)**2
                weighted_mean2 = np.sum(weights2 * slopes2) / np.sum(weights2)
                weighted_mean_error2 = np.sqrt(1 / np.sum(weights2))
                weighted_variance2 = np.sum(weights2 * (slopes2 - weighted_mean2)**2) / np.sum(weights2)
                weighted_std_dev2 = np.sqrt(weighted_variance2)
                intervariability2 = np.std(slopes2, ddof=1)  # Standard deviation (sample)
                combined_sigma2 = np.sqrt(weighted_std_dev2**2+intervariability2**2)

                ringwog=[slopes[0],slopes[1],slopes[2],slopes[3],slopes[4],slopes[5],slopes[6],slopes[7],slopes[8],slopes[9],slopes[10],slopes[11],slopes[12],slopes[13],slopes[14],slopes[15]]
                ringwg=[slopes[16],slopes[17],slopes[18],slopes[19]]
                
                meanwog = np.mean(ringwog)
                meanwg = np.mean(ringwg)
                gain = weighted_mean2/weighted_mean1
                deviationwog = np.std(ringwog)
                deviationwg = np.std(ringwg)
               
                gainerror=gain*math.sqrt(weighted_std_dev2/(weighted_mean2**2)+weighted_std_dev1/(weighted_mean1**2))
                print('--------------------------------------------------------------------------------------------')
                print(f'Mean characterization ring 0-3, {wl} = {meanwog} +- {deviationwog}. The error in percent is: {100*deviationwog/meanwog}$%$')
                print(f'Weighted mean characterization ring 0-3, {wl} = {weighted_mean1} +- {weighted_mean_error1}. The error in percent is: {100*weighted_mean_error1/weighted_mean1}$%$')
                print(f'Weighted standard deviation (spread): {weighted_std_dev1}')
                print(f'Standard deviation of the measurements, intervariability: {intervariability1}')
                print(f'Total combined error:{combined_sigma1}')
                print(f'=> Weighted characterization ring 0-3, {wl} = {weighted_mean1} +- {combined_sigma1}. The error in percent is: {100*combined_sigma1/weighted_mean1}$%$')

                
                print(f'Mean characterization ring 4 {wl} = {meanwg} +- {deviationwg}. The error in percent is: {100*deviationwg/meanwg}$%$')
                print(f'Weighted mean characterization ring 4, {wl} = {weighted_mean2} +- {weighted_mean_error2}. The error in percent is: {100*weighted_mean_error2/weighted_mean2}$%$')
                print(f'Weighted standard deviation (spread): {weighted_std_dev2}')
                print(f'Standard deviation of the measurements, intervariability: {intervariability2}')
                print(f'Total combined error:{combined_sigma2}')
                print(f'=> Weighted characterization ring 4, {wl} = {weighted_mean2} +- {combined_sigma2}. The error in percent is: {100*combined_sigma2/weighted_mean2}$%$')
                
                print(f"==> Gain for ring 4 vs. 1-3 from weighted mean,{wl} = {gain} +- {gainerror}. The error in percent is: {100*gainerror/gain}$%$")

                print('--------------------------------------------------------------------------------------------')


                fig = plt.figure(101)
                plt.errorbar(range(len(slopes1)),slopes1,slopes_stderr1,fmt='.',color='tab:purple',markersize=10,linewidth=1)
                plt.axhline(y=weighted_mean1, color='tab:olive', linestyle='-', label='wmean ADC/V')
                plt.fill_between(range(-1,len(slopes1)+1), (weighted_mean1-weighted_std_dev1), (weighted_mean1+weighted_std_dev1), color='tab:olive', alpha=0.2, label= '${\sigma}_{weightStd}$')
                #plt.fill_between(range(-1,len(slopes1)+1), (weighted_mean1-combined_sigma1), (weighted_mean1+combined_sigma1), color='tab:cyan', alpha=0.2, label= '${\sigma}_{weightStd} + {\sigma}_{intervar} $')
                # plt.axhline(y=np.mean(slopes1), color='tab:olive', linestyle='-', label='mean ADC/V')
                # plt.fill_between(range(-1,len(slopes1)+1), (np.mean(slopes1)-np.std(slopes1)), (np.mean(slopes1)+np.std(slopes1)), color='tab:olive', alpha=0.2)
                plt.title(f'Sensors char at {wl} nm. Value =  {weighted_mean1:.1f} +- {combined_sigma1:.1f} ADC/V ({100*combined_sigma1/weighted_mean1:.1f}%)')
                plt.xlabel('SensorID')
                plt.ylabel('ADC counts/V')
                plt.xticks(np.arange(len(slopes1)), sensorsID1, rotation=20)
                plt.xlim([-1,len(sensorsID1)]) 
                plt.grid()
                plt.legend()
                plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_calibration_0_3.png',dpi=199)  # Display the current figure
                plt.close()
                
                if(lowLevel_plot):

                    fig = plt.figure(200)
                    plt.errorbar(range(len(slopes_stderr1)), slopes_stderr1, fmt='.',color='tab:purple', markersize=10, linewidth=1)
                    plt.axhline(y=np.mean(slopes_stderr1), color='tab:olive', linestyle='-', label='mean ADC/V')
                    plt.fill_between(range(-1,len(slopes_stderr1)+1), (np.mean(slopes_stderr1)-np.std(slopes_stderr1)), (np.mean(slopes_stderr1)+np.std(slopes_stderr1)), color='tab:olive', alpha=0.2)
                    plt.title(f'Stderr at {wl} nm. Value = {np.mean(slopes_stderr1):.1f} +- {np.std(slopes_stderr1):.1f}')
                    plt.xlabel('SensorID')
                    plt.ylabel('ADC counts/V')
                    plt.xticks(np.arange(len(slopes_stderr1)), sensorsID1, rotation=20)
                    plt.xlim([-1,len(sensorsID1)])
                    plt.grid()
                    plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_calibration_stderrs_0_3.png',dpi=199)  # Display the current figure
                    plt.close()

                fig = plt.figure(102)
                plt.errorbar(range(len(slopes2)), slopes2, slopes_stderr2, fmt='.',color='tab:purple', markersize=10, linewidth=1)
                plt.axhline(y=weighted_mean2, color='tab:olive', linestyle='-', label='wmean ADC/V')
                plt.fill_between(range(-1,len(slopes2)+1), (weighted_mean2-weighted_std_dev2), (weighted_mean2+weighted_std_dev2), color='tab:olive', alpha=0.2, label= '${\sigma}_{weightStd} $')
                #plt.fill_between(range(-1,len(slopes2)+1), (weighted_mean2-combined_sigma2), (weighted_mean2+combined_sigma2), color='tab:cyan', alpha=0.2,label= '${\sigma}_{weightStd} + {\sigma}_{intervar} $')
                #plt.axhline(y=np.mean(slopes2), color='tab:olive', linestyle='-', label='mean ADC/V')
                #plt.fill_between(range(-1,len(slopes2)+1), (np.mean(slopes2)-np.std(slopes2)), (np.mean(slopes2)+np.std(slopes2)), color='tab:olive', alpha=0.2)
                plt.title(f'Sensors char at {wl} nm. Value = {weighted_mean2:.1f} +- {combined_sigma2:.1f} ADC/V ({100*combined_sigma2/weighted_mean2:.1f}%)')
                plt.xlabel('SensorID')
                plt.ylabel('ADC counts/V')
                plt.xticks(np.arange(len(slopes2)), sensorsID2, rotation=20)
                plt.xlim([-1,len(sensorsID2)]) 
                plt.grid()
                plt.legend()
                plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_calibration_4.png',dpi=199)  # Display the current figure
                plt.close()
                
                if(lowLevel_plot):
                    fig = plt.figure(200)
                    plt.errorbar(range(len(slopes_stderr2)), slopes_stderr2, fmt='.', color='tab:purple', markersize=10, linewidth=1)
                    plt.axhline(y=np.mean(slopes_stderr2), color='tab:olive', linestyle='-', label='mean ADC/V')
                    plt.fill_between(range(-1,len(slopes_stderr2)+1), (np.mean(slopes_stderr2)-np.std(slopes_stderr2)), (np.mean(slopes_stderr2)+np.std(slopes_stderr2)), color='tab:olive', alpha=0.2)
                    plt.title(f'Stderr at {wl} nm. Value = {np.mean(slopes_stderr2):.1f} +- {np.std(slopes_stderr2):.1f}')
                    plt.xlabel('SensorID')
                    plt.ylabel('ADC counts/V')
                    plt.xticks(np.arange(len(slopes_stderr2)), sensorsID2, rotation=20)
                    plt.xlim([-1,len(sensorsID2)])
                    plt.grid()
                    plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_calibration_stderrs_4.png',dpi=199)  # Display the current figure
                    plt.close()
        if(pedestal_plot):
            fig =  plt.figure(11)
            plt.errorbar(range(len(pedestalsADC)),pedestalsADC, pedestalsADC_stderr, fmt='.',color='tab:purple', markersize=10, linewidth=1)
            plt.xlabel('SensorID')
            plt.ylabel('ADC counts')
            plt.xticks(range(len(pedestalsADC)), sensorsID, rotation=20)
            plt.xlim([-1,len(sensorsID)])
            plt.grid()
            plt.title(f'Pedestal of the ADC for measurement of {wl} laser')
            plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_pedestals_ADC.png',dpi=199)  # Display the current figure
            plt.close()
            
            fig =  plt.figure(12)
            plt.errorbar(range(len(pedestalsRefPD)),pedestalsRefPD, pedestalsRefPD_stderr, fmt='.',color='tab:purple', markersize=10, linewidth=1)
            plt.xlabel('SensorID')
            plt.ylabel('V')
            plt.xticks(range(len(pedestalsADC)), sensorsID, rotation=20)
            plt.xlim([-1,len(sensorsID)])
            plt.grid()
            plt.title(f'Pedestals of the PD reference for measurement of {wl} laser')
            plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_pedestals_RefPD.png',dpi=199)  # Display the current figure
            plt.close()

In [10]:
def ADC_analysis_sensor(folder_path,dataset_number,output_path,output_path_lowlevel,datetest,boardname):

    outputpath = f"{output_path}"
    outputpath_lowlevel = f"{output_path_lowlevel}"

    #sensorsID = ['0.0','0.1','0.2','0.3','1.0','1.1','1.2','1.3','2.0','2.1','2.2','2.3','3.0','3.1','3.2','3.3','4.0','4.1','4.2','4.3']
    #sensorsID = ['0.0','0.1','1.0','1.1','2.0','2.1','3.0','3.1','4.0','4.1']
    sensorsID = ['0.2','0.3','1.2','1.3','2.2','2.3','3.2','3.3','4.2','4.3']
    #sensorsID = ['4.2']
    wavelength = ['1064','532']
   
    if not os.path.exists(outputpath_lowlevel):
        os.makedirs(outputpath_lowlevel)
        os.makedirs(outputpath)
        print(f"Directory '{outputpath_lowlevel}' created.")

    if not os.path.exists(outputpath):
        os.makedirs(outputpath)
        print(f"Directory '{outputpath}' created.")

    for wl in wavelength:
        pedestalsADC = []
        pedestalsADC_stderr = []
        pedestalsRefPD = []
        pedestalsRefPD_stderr = []
        slopes = []
        intercepts= []
        rCoes = []
        slopes_stderr = []
        intercepts_stderr = []
        saturationADCs = []
        if wl== '532':
        # Rename DataFrame columns
            strg_L = 'Laser Current (mA)'
            #temp = '20'
        elif wl == '1064':
            strg_L = 'Laser Power (mW)'
            #temp = '25'

        for l in sensorsID:
            sensorID=l
            pedestal_list = []
            main_list = []
            for number in np.arange(1,dataset_number+1):
                filename = f'{folder_path}/{datetest}_{boardname}_{sensorID}_{wl}_{number}.txt'
                # Read the first line as a separate DataFrame
                df = pd.read_csv(filename, sep="\t", header=None)
                
                # Split the first row into a separate DataFrame
                first_row = df.iloc[[0]]  # Select the first row as a DataFrame
                last_row = df.iloc[[-1]]
                pedestal_list.append(first_row)  # Add the first row to pedestal list
                pedestal_list.append(last_row)  # Add the first row to pedestal list

                # Get the remaining rows (from the second row onward)
                main_data = df.iloc[1:-1]  # Select all rows except the first and last
                main_list.append(main_data)  # Add to main data list

            # Concatenate the first rows (metadata or header info) and the main data
            TP = pd.concat(pedestal_list, ignore_index=True)
            T = pd.concat(main_list, ignore_index=True)
            TP.columns = ['Date-Hour', 'L', 'TotalSum', 'TotalSquareSum', 'meanRefPD', 'stdRefPD', 'Tem', 'RH', 'TotalCounts']
            T.columns = ['Date-Hour', 'L', 'TotalSum', 'TotalSquareSum', 'meanRefPD', 'stdRefPD', 'Tem', 'RH', 'TotalCounts']

            meanADC = T['TotalSum']/T['TotalCounts']
            stdADC = np.sqrt((T['TotalSquareSum']-T['TotalCounts']*meanADC**2)/(T['TotalCounts']-1))
            T['meanADC'] = meanADC
            T['stdADC'] = stdADC
            
            pedestal_meanADC = TP['TotalSum']/TP['TotalCounts']
            TP['meanADC'] = pedestal_meanADC
            TP['stdADC'] = np.sqrt((TP['TotalSquareSum']-TP['TotalCounts']*pedestal_meanADC**2)/(TP['TotalCounts']-1))

            # meanPM = meanADC*0.61e-3 #0.61mV/ADCcount  
            # stdPM = stdADC*0.61e-3
            # # T['meanPM']=meanPM
            # T['stdPM']=stdPM
            print(f'Starting the fitting procedure for calibration coefficient for sensorID {sensorID} for laser {wl} nm')


            if fit:
                T, saturationADC = find_saturation_from_derivate(T)
                #saturationADC=-1
                res = linregress(T['meanRefPD'], T['meanADC'])
                
                slopes.append(res.slope)
                intercepts.append(res.intercept)
                rCoes.append(res.rvalue)
                slopes_stderr.append(res.stderr)
                intercepts_stderr.append(res.intercept_stderr)
                saturationADCs.append(saturationADC)
            
            pedestalsADC_wmean, pedestalsADC_wstderr = ObtainMeanAndStd(TP['meanADC'],TP['stdADC'])
            pedestalsPD_wmean, pedestalsPD_wstderr = ObtainMeanAndStd(TP['meanRefPD'],TP['stdRefPD'])


            pedestalsADC.append(pedestalsADC_wmean)
            #sqrt((∑X^2-N*meanADC^2/(N-1))
            pedestalsADC_stderr.append(pedestalsADC_wstderr) # This errors should be added in quadrature
            pedestalsRefPD.append(pedestalsPD_wmean) 
            pedestalsRefPD_stderr.append(pedestalsPD_wstderr)# This errors should be added in quadrature


            if(verbose and fit):
                print(f'Calibration coefficient found for sensorID {sensorID} for laser {wl} nm = {res.slope} +/- {res.stderr} in ADC/V')

            if(lowLevel_plot):
                fig = plt.figure(2)
                plt.errorbar(T['L'], T['meanADC'], yerr=T['stdADC'], fmt='.', markersize=10, linewidth=1)
                plt.ylabel('mean ADC counts')
                plt.xlabel(strg_L)
                plt.grid()
                plt.title(f'Plot sensorID {sensorID}, wavelength {wl} nm')
                plt.tight_layout()
                plt.savefig(f'{outputpath_lowlevel}/{datetest}_{boardname}_{sensorID}_{wl}_meanADC_L.png',dpi=199)  # Display the current figure
                plt.close(fig)

                fig = plt.figure(4)
                plt.errorbar(T['L'], T['meanRefPD'], yerr=T['stdRefPD'], fmt='.', markersize=10, linewidth=1)
                plt.ylabel('Mean ref PD (V)')
                plt.xlabel(strg_L)
                plt.grid()
                plt.title(f'Plot sensorID {sensorID}, wavelength {wl} nm')
                plt.tight_layout()
                plt.savefig(f'{outputpath_lowlevel}/{datetest}_{boardname}_{sensorID}_{wl}_meanRefPD_L.png',dpi=199)  # Display the current figure
                #plt.show()
                plt.close(fig)

                #the label here are incorrect since meanPM=meanADC*0.61e-3 #0.61mV/ADCcoun. Monica proposed from the baffle paper: 
                #Voltage/ADCcount =3.3V/2**10=0.00322 V/ADCcount
                #4.6 uW/ADCcount*1ADCcount/0.00322V = 1428 uW/V

                if fit:
                    fig = plt.figure(6)
                    plt.errorbar(T['meanRefPD'], T['meanADC'], yerr=T['stdADC'], fmt='.', markersize=10, linewidth=1)
                    plt.plot(T['meanRefPD'], res.intercept + res.slope*T['meanRefPD'], 'r', label='fitted line')
                    plt.ylabel('Mean ADC counts')
                    plt.xlabel('Mean ref PD (V)')
                    plt.grid()
                    plt.title(f'Plot sensorID {sensorID}, wavelength {wl} nm')
                    plt.tight_layout()
                    plt.savefig(f'{outputpath_lowlevel}/{datetest}_{boardname}_{sensorID}_{wl}_meanADC_meanRefPD.png',dpi=199)  # Display the current figure
                    #plt.show()
                    plt.close(fig)
            
        if(highLevel_plot and fit):
                tabdata = pd.DataFrame()
                tabdata["sensorID"] = sensorsID
                tabdata["slope"] = slopes
                tabdata["intercept"] = intercepts
                tabdata["rCoe"] = rCoes
                tabdata["slopes_stderr"] = slopes_stderr
                tabdata["intercept_stderr"] = intercepts_stderr
                tabdata["saturationADC"] = saturationADCs
                
                fig = plt.figure(300)
                plt.errorbar(range(len(intercepts)), intercepts,intercepts_stderr, fmt='.',  markersize=10, linewidth=1)
                plt.axhline(y=np.mean(intercepts), color='tab:olive', linestyle='-')
                plt.fill_between(range(-1,len(intercepts)+1), (np.mean(intercepts)-np.std(intercepts)), (np.mean(intercepts)+np.std(intercepts)), color='tab:olive', alpha=0.2)
                plt.title(f'Intercepts at {wl} nm. Value = {np.mean(intercepts):.1f} +- {np.std(intercepts):.1f}')
                plt.xlabel('SensorID')
                plt.ylabel('ADC counts')
                plt.xticks(np.arange(len(intercepts)), sensorsID, rotation=20)
                plt.xlim([-1,len(sensorsID)])
                plt.grid()
                plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_noshade_intercepts.png',dpi=199)  # Display the current figure
                plt.close()
                
                fig = plt.figure(400)
                plt.errorbar(range(len(rCoes)), rCoes, fmt='.', markersize=10,linewidth=1)
                plt.axhline(y=np.mean(rCoes), color='tab:olive', linestyle='-')
                plt.fill_between(range(-1,len(rCoes)+1), (np.mean(rCoes)-np.std(rCoes)), (np.mean(rCoes)+np.std(rCoes)), color='tab:olive', alpha=0.2)
                plt.title(f'rCoes at {wl} nm. Value = {np.mean(rCoes):.1f} +- {np.std(rCoes):.1f}')
                plt.xlabel('SensorID')
                plt.ylabel('')
                plt.xticks(np.arange(len(rCoes)), sensorsID, rotation=20)
                plt.xlim([-1,len(sensorsID)])
                plt.grid()
                plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_rcoes.png',dpi=199)  # Display the current figure
                plt.close()
                
                fig = plt.figure(500)
                plt.title(f'Saturation ADC at {wl} nm. Value = {np.mean(saturationADCs):.1f} +- {np.std(saturationADCs):.1f}')
                plt.plot(range(len(saturationADCs)), saturationADCs,'.')
                plt.axhline(y=np.mean(saturationADCs), color='tab:olive', linestyle='-')
                plt.fill_between(range(-1,len(saturationADCs)+1), (np.mean(saturationADCs)-np.std(saturationADCs)), (np.mean(saturationADCs)+np.std(saturationADCs)), color='tab:olive', alpha=0.2)
                plt.xlabel('SensorID')
                plt.ylabel('')
                plt.xticks(np.arange(len(saturationADCs)), sensorsID, rotation=20)
                plt.xlim([-1,len(sensorsID)])
                plt.grid()
                plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_saturationADC.png',dpi=199)  # Display the current figure
                plt.close()

                #Uncertainty of Weighted Mean: Precision based on individual measurement uncertainties.
                #Intervariability: Variability among the measurements (random error, i.e. embedded systematic uncertainties).
                #Purely Systematic Error: Accounts for known or estimated biases in the measurements (temperature/humidity????)

                weights = 1 / np.array(slopes_stderr)**2
                weighted_mean = np.sum(weights * slopes) / np.sum(weights)
                weighted_mean_error = np.sqrt(1 / np.sum(weights))
                weighted_variance = np.sum(weights * (slopes - weighted_mean)**2) / np.sum(weights)
                weighted_std_dev = np.sqrt(weighted_variance)
                intervariability = np.std(slopes, ddof=1)  # Standard deviation (sample)
                combined_sigma = np.sqrt(weighted_std_dev**2+intervariability**2)
                
                #ringwog=[slopes[0],slopes[1],slopes[2],slopes[3],slopes[4],slopes[5],slopes[6],slopes[7],slopes[8],slopes[9],slopes[10],slopes[11],slopes[12],slopes[13],slopes[14],slopes[15]]
                #ringwg=[slopes[16],slopes[17],slopes[18],slopes[19]]
                
                mean = np.mean(slopes)
                std = np.std(slopes)
               
                print('--------------------------------------------------------------------------------------------')
                print(f'Mean characterization sensors, {wl} = {mean} +- {std}. The error in percent is: {100*std/mean}$%$')
                print(f'Weighted mean sensors, {wl} = {weighted_mean} +- {weighted_mean_error}. The error in percent is: {100*weighted_mean_error/weighted_mean}$%$')
                print(f'Weighted standard deviation (spread): {weighted_std_dev}')
                print(f'Standard deviation of the measurements, intervariability: {intervariability}')
                print(f'Total combined error:{combined_sigma}')
                print(f'=> Weighted characterization sensor, {wl} = {weighted_mean} +- {combined_sigma}. The error in percent is: {100*combined_sigma/weighted_mean}$%$')

                
                print('-------------------------------------------------------------------------------------------')


                fig = plt.figure(101)
                plt.errorbar(range(len(slopes)),slopes,slopes_stderr,fmt='.',color='tab:purple',markersize=10,linewidth=1)
                plt.axhline(y=weighted_mean, color='tab:olive', linestyle='-', label='wmean ADC/V')
                plt.fill_between(range(-1,len(slopes)+1), (weighted_mean-weighted_std_dev), (weighted_mean+weighted_std_dev), color='tab:olive', alpha=0.2, label= '${\sigma}_{weightStd}$')
                #plt.fill_between(range(-1,len(slopes1)+1), (weighted_mean1-combined_sigma1), (weighted_mean1+combined_sigma1), color='tab:cyan', alpha=0.2, label= '${\sigma}_{weightStd} + {\sigma}_{intervar} $')
                # plt.axhline(y=np.mean(slopes1), color='tab:olive', linestyle='-', label='mean ADC/V')
                # plt.fill_between(range(-1,len(slopes1)+1), (np.mean(slopes1)-np.std(slopes1)), (np.mean(slopes1)+np.std(slopes1)), color='tab:olive', alpha=0.2)
                plt.title(f'Sensors char at {wl} nm. Value =  {weighted_mean:.1f} +- {combined_sigma:.1f} ADC/V ({100*combined_sigma/weighted_mean:.1f}%)')
                plt.xlabel('SensorID')
                plt.ylabel('ADC counts/V')
                plt.xticks(np.arange(len(slopes)), sensorsID, rotation=20)
                plt.xlim([-1,len(sensorsID)]) 
                plt.grid()
                plt.legend()
                plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_calibration.png',dpi=199)  # Display the current figure
                plt.close()
                
                if(lowLevel_plot):

                    fig = plt.figure(200)
                    plt.errorbar(range(len(slopes_stderr)), slopes_stderr, fmt='.',color='tab:purple', markersize=10, linewidth=1)
                    plt.axhline(y=np.mean(slopes_stderr), color='tab:olive', linestyle='-', label='mean ADC/V')
                    plt.fill_between(range(-1,len(slopes_stderr)+1), (np.mean(slopes_stderr)-np.std(slopes_stderr)), (np.mean(slopes_stderr)+np.std(slopes_stderr)), color='tab:olive', alpha=0.2)
                    plt.title(f'Stderr at {wl} nm. Value = {np.mean(slopes_stderr):.1f} +- {np.std(slopes_stderr):.1f}')
                    plt.xlabel('SensorID')
                    plt.ylabel('ADC counts/V')
                    plt.xticks(np.arange(len(slopes_stderr)), sensorsID, rotation=20)
                    plt.xlim([-1,len(sensorsID)])
                    plt.grid()
                    plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_calibration_stderrs_0_3.png',dpi=199)  # Display the current figure
                    plt.close()

        if(pedestal_plot):
            fig =  plt.figure(11)
            plt.errorbar(range(len(pedestalsADC)),pedestalsADC, pedestalsADC_stderr, fmt='.',color='tab:purple', markersize=10, linewidth=1)
            plt.xlabel('SensorID')
            plt.ylabel('ADC counts')
            plt.xticks(range(len(pedestalsADC)), sensorsID, rotation=20)
            plt.xlim([-1,len(sensorsID)])
            plt.grid()
            plt.title(f'Pedestal of the ADC for measurement of {wl} laser')
            plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_pedestals_ADC.png',dpi=199)  # Display the current figure
            plt.close()
            
            fig =  plt.figure(12)
            plt.errorbar(range(len(pedestalsRefPD)),pedestalsRefPD, pedestalsRefPD_stderr, fmt='.',color='tab:purple', markersize=10, linewidth=1)
            plt.xlabel('SensorID')
            plt.ylabel('V')
            plt.xticks(range(len(pedestalsADC)), sensorsID, rotation=20)
            plt.xlim([-1,len(sensorsID)])
            plt.grid()
            plt.title(f'Pedestals of the PD reference for measurement of {wl} laser')
            plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_pedestals_RefPD.png',dpi=199)  # Display the current figure
            plt.close()

In [18]:
def ADC_analysis_sensor_gain(folder_path,dataset_number,output_path,output_path_lowlevel,datetest,boardname):

    outputpath = f"{output_path}"
    outputpath_lowlevel = f"{output_path_lowlevel}"
    
    #wavelength = ['532']
    #filterOrder= ['FW5', 'FW5','FW5','FW5','FW5','FW5','FW5','FW5','FW5','FW5'] 
    #filterOrder= ['FW4', 'FW4','FW4','FW4','FW4','FW4','FW5','FW5','FW5','FW5'] 
    #sensorsID = ['0.0','0.1','1.0','1.1','2.0','2.1','3.0','3.1','4.0','4.1']
    #sensorsID = ['0.2', '0.3','1.2', '1.3','2.2', '2.3', '3.2', '3.3', '4.2', '4.3']



    wavelength = ['532']
    filterOrder= ['FW4'] 
    #sensorsID = ['0.0','0.1','1.0','1.1','2.0','2.1','3.0','3.1','4.0','4.1']
    sensorsID = ['0.2']
    #filterOrder= ['FW5', 'FW5','FW5','FW5','FW5','FW5','FW5','FW5','FW5','FW5','FW5', 'FW5','FW5','FW5','FW5','FW5','FW5','FW5','FW5','FW5'] 
    #sensorsID = ['0.0','0.1','0.2', '0.3','1.0','1.1','1.2', '1.3','2.0','2.1','2.2', '2.3']#,'3.0','3.1', '3.2', '3.3', '4.0','4.1','4.2', '4.3']


   
    if not os.path.exists(outputpath_lowlevel):
        os.makedirs(outputpath_lowlevel)
        os.makedirs(outputpath)
        print(f"Directory '{outputpath_lowlevel}' created.")

    if not os.path.exists(outputpath):
        os.makedirs(outputpath)
        print(f"Directory '{outputpath}' created.")

    for wl in wavelength:
        pedestalsADC = []
        pedestalsADC_stderr = []
        pedestalsRefPD = []
        pedestalsRefPD_stderr = []
        slopes = []
        intercepts= []
        rCoes = []
        slopes_stderr = []
        intercepts_stderr = []
        saturationADCs = []
        if wl== '532':
        # Rename DataFrame columns
            strg_L = 'Laser Current (mA)'
            #temp = '20'
        elif wl == '1064':
            strg_L = 'Laser Power (mW)'
            #temp = '25'

        for l, sensorID in enumerate(sensorsID):
            filter = filterOrder[l]
            pedestal_list = []
            main_list = []
            for number in np.arange(1,dataset_number+1):
                filename = f'{folder_path}/{datetest}_{boardname}_{sensorID}_{wl}_{filter}_{number}.txt'
                # Read the first line as a separate DataFrame
                df = pd.read_csv(filename, sep="\t", header=None)
                
                # Split the first row into a separate DataFrame
                first_row = df.iloc[[0]]  # Select the first row as a DataFrame
                last_row = df.iloc[[-1]]
                pedestal_list.append(first_row)  # Add the first row to pedestal list
                pedestal_list.append(last_row)  # Add the first row to pedestal list

                # Get the remaining rows (from the second row onward)
                main_data = df.iloc[1:-1]  # Select all rows except the first and last
                main_list.append(main_data)  # Add to main data list

            # Concatenate the first rows (metadata or header info) and the main data
            TP = pd.concat(pedestal_list, ignore_index=True)
            T = pd.concat(main_list, ignore_index=True)
            TP.columns = ['Date-Hour', 'L', 'TotalSum', 'TotalSquareSum', 'meanRefPD', 'stdRefPD', 'Tem', 'RH', 'TotalCounts']
            T.columns = ['Date-Hour', 'L', 'TotalSum', 'TotalSquareSum', 'meanRefPD', 'stdRefPD', 'Tem', 'RH', 'TotalCounts']

            meanADC = T['TotalSum']/T['TotalCounts']
            stdADC = np.sqrt((T['TotalSquareSum']-T['TotalCounts']*meanADC**2)/(T['TotalCounts']-1))
            T['meanADC'] = meanADC
            T['stdADC'] = stdADC 

            pedestal_meanADC = TP['TotalSum']/TP['TotalCounts']
            TP['meanADC'] = pedestal_meanADC
            TP['stdADC'] = np.sqrt((TP['TotalSquareSum']-TP['TotalCounts']*pedestal_meanADC**2)/(TP['TotalCounts']-1))

            # meanPM = meanADC*0.61e-3 #0.61mV/ADCcount  
            # stdPM = stdADC*0.61e-3
            # # T['meanPM']=meanPM
            # T['stdPM']=stdPM
            print(f'Starting the fitting procedure for calibration coefficient for sensorID {sensorID} for laser {wl} nm')


            if fit:
                T, saturationADC = find_saturation_from_derivate(T)
                #saturationADC=-1
                res = linregress(T['meanRefPD'], T['meanADC'])
                
                slopes.append(res.slope)
                intercepts.append(res.intercept)
                rCoes.append(res.rvalue)
                slopes_stderr.append(res.stderr)
                intercepts_stderr.append(res.intercept_stderr)
                saturationADCs.append(saturationADC)
            
            pedestalsADC_wmean, pedestalsADC_wstderr = ObtainMeanAndStd(TP['meanADC'],TP['stdADC'])
            pedestalsPD_wmean, pedestalsPD_wstderr = ObtainMeanAndStd(TP['meanRefPD'],TP['stdRefPD'])


            pedestalsADC.append(pedestalsADC_wmean)
            #sqrt((∑X^2-N*meanADC^2/(N-1))
            pedestalsADC_stderr.append(pedestalsADC_wstderr) # This errors should be added in quadrature
            pedestalsRefPD.append(pedestalsPD_wmean) 
            pedestalsRefPD_stderr.append(pedestalsPD_wstderr)# This errors should be added in quadrature


            if(verbose and fit):
                print(f'Calibration coefficient found for sensorID {sensorID} for laser {wl} nm = {res.slope} +/- {res.stderr} in ADC/V')

            if(lowLevel_plot):
                fig = plt.figure(2)
                plt.errorbar(T['L'], T['meanADC'], yerr=T['stdADC'], fmt='.', markersize=10, linewidth=1)
                plt.ylabel('mean ADC counts')
                plt.xlabel(strg_L)
                plt.grid()
                plt.title(f'Plot sensorID {sensorID}, wavelength {wl} nm, filter = {filter}')
                plt.tight_layout()
                plt.savefig(f'{outputpath_lowlevel}/{datetest}_{boardname}_{sensorID}_{wl}_meanADC_L.png',dpi=199)  # Display the current figure
                plt.close(fig)

                fig = plt.figure(4)
                plt.errorbar(T['L'], T['meanRefPD'], yerr=T['stdRefPD'], fmt='.', markersize=10, linewidth=1)
                plt.ylabel('Mean ref PD (V)')
                plt.xlabel(strg_L)
                plt.grid()
                plt.title(f'Plot sensorID {sensorID}, wavelength {wl} nm, filter = {filter}')
                plt.tight_layout()
                plt.savefig(f'{outputpath_lowlevel}/{datetest}_{boardname}_{sensorID}_{wl}_meanRefPD_L.png',dpi=199)  # Display the current figure
                #plt.show()
                plt.close(fig)

                #the label here are incorrect since meanPM=meanADC*0.61e-3 #0.61mV/ADCcoun. Monica proposed from the baffle paper: 
                #Voltage/ADCcount =3.3V/2**10=0.00322 V/ADCcount
                #4.6 uW/ADCcount*1ADCcount/0.00322V = 1428 uW/V

                if fit:
                    fig = plt.figure(6)
                    plt.errorbar(T['meanRefPD'], T['meanADC'], yerr=T['stdADC'], fmt='.', markersize=10, linewidth=1)
                    plt.plot(T['meanRefPD'], res.intercept + res.slope*T['meanRefPD'], 'r', label='fitted line')
                    plt.ylabel('Mean ADC counts')
                    plt.xlabel('Mean ref PD (V)')
                    plt.grid()
                    plt.title(f'Plot sensorID {sensorID}, wavelength {wl} nm, filter = {filter}')
                    plt.tight_layout()
                    plt.savefig(f'{outputpath_lowlevel}/{datetest}_{boardname}_{sensorID}_{wl}_meanADC_meanRefPD.png',dpi=199)  # Display the current figure
                    #plt.show()
                    plt.close(fig)
            
        if(highLevel_plot and fit):
                tabdata = pd.DataFrame()
                tabdata["sensorID"] = sensorsID
                tabdata["slope"] = slopes
                tabdata["intercept"] = intercepts
                tabdata["rCoe"] = rCoes
                tabdata["slopes_stderr"] = slopes_stderr
                tabdata["intercept_stderr"] = intercepts_stderr
                tabdata["saturationADC"] = saturationADCs
                
                fig = plt.figure(300)
                plt.errorbar(range(len(intercepts)), intercepts,intercepts_stderr, fmt='.',  markersize=10, linewidth=1)
                plt.axhline(y=np.mean(intercepts), color='tab:olive', linestyle='-')
                plt.fill_between(range(-1,len(intercepts)+1), (np.mean(intercepts)-np.std(intercepts)), (np.mean(intercepts)+np.std(intercepts)), color='tab:olive', alpha=0.2)
                plt.title(f'Intercepts at {wl} nm. Value = {np.mean(intercepts):.1f} +- {np.std(intercepts):.1f}')
                plt.xlabel('SensorID')
                plt.ylabel('ADC counts')
                plt.xticks(np.arange(len(intercepts)), sensorsID, rotation=20)
                plt.xlim([-1,len(sensorsID)])
                plt.grid()
                plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_noshade_intercepts.png',dpi=199)  # Display the current figure
                plt.close()
                
                fig = plt.figure(400)
                plt.errorbar(range(len(rCoes)), rCoes, fmt='.', markersize=10,linewidth=1)
                plt.axhline(y=np.mean(rCoes), color='tab:olive', linestyle='-')
                plt.fill_between(range(-1,len(rCoes)+1), (np.mean(rCoes)-np.std(rCoes)), (np.mean(rCoes)+np.std(rCoes)), color='tab:olive', alpha=0.2)
                plt.title(f'rCoes at {wl} nm. Value = {np.mean(rCoes):.1f} +- {np.std(rCoes):.1f}')
                plt.xlabel('SensorID')
                plt.ylabel('')
                plt.xticks(np.arange(len(rCoes)), sensorsID, rotation=20)
                plt.xlim([-1,len(sensorsID)])
                plt.grid()
                plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_rcoes.png',dpi=199)  # Display the current figure
                plt.close()
                
                fig = plt.figure(500)
                plt.title(f'Saturation ADC at {wl} nm. Value = {np.mean(saturationADCs):.1f} +- {np.std(saturationADCs):.1f}')
                plt.plot(range(len(saturationADCs)), saturationADCs,'.')
                plt.axhline(y=np.mean(saturationADCs), color='tab:olive', linestyle='-')
                plt.fill_between(range(-1,len(saturationADCs)+1), (np.mean(saturationADCs)-np.std(saturationADCs)), (np.mean(saturationADCs)+np.std(saturationADCs)), color='tab:olive', alpha=0.2)
                plt.xlabel('SensorID')
                plt.ylabel('')
                plt.xticks(np.arange(len(saturationADCs)), sensorsID, rotation=20)
                plt.xlim([-1,len(sensorsID)])
                plt.grid()
                plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_saturationADC.png',dpi=199)  # Display the current figure
                plt.close()

                #Uncertainty of Weighted Mean: Precision based on individual measurement uncertainties.
                #Intervariability: Variability among the measurements (random error, i.e. embedded systematic uncertainties).
                #Purely Systematic Error: Accounts for known or estimated biases in the measurements (temperature/humidity????)

                weights = 1 / np.array(slopes_stderr)**2
                weighted_mean = np.sum(weights * slopes) / np.sum(weights)
                weighted_mean_error = np.sqrt(1 / np.sum(weights))
                weighted_variance = np.sum(weights * (slopes - weighted_mean)**2) / np.sum(weights)
                weighted_std_dev = np.sqrt(weighted_variance)
                intervariability = np.std(slopes, ddof=1)  # Standard deviation (sample)
                combined_sigma = np.sqrt(weighted_std_dev**2+intervariability**2)
                
                #ringwog=[slopes[0],slopes[1],slopes[2],slopes[3],slopes[4],slopes[5],slopes[6],slopes[7],slopes[8],slopes[9],slopes[10],slopes[11],slopes[12],slopes[13],slopes[14],slopes[15]]
                #ringwg=[slopes[16],slopes[17],slopes[18],slopes[19]]
                
                mean = np.mean(slopes)
                std = np.std(slopes)
               
                print('--------------------------------------------------------------------------------------------')
                print(f'Mean characterization sensors, {wl} = {mean} +- {std}. The error in percent is: {100*std/mean}$%$')
                print(f'Weighted mean sensors, {wl} = {weighted_mean} +- {weighted_mean_error}. The error in percent is: {100*weighted_mean_error/weighted_mean}$%$')
                print(f'Weighted standard deviation (spread): {weighted_std_dev}')
                print(f'Standard deviation of the measurements, intervariability: {intervariability}')
                print(f'Total combined error:{combined_sigma}')
                print(f'=> Weighted characterization sensor, {wl} = {weighted_mean} +- {combined_sigma}. The error in percent is: {100*combined_sigma/weighted_mean}$%$')

                
                print('-------------------------------------------------------------------------------------------')


                fig = plt.figure(101)
                plt.errorbar(range(len(slopes)),slopes,slopes_stderr,fmt='.',color='tab:purple',markersize=10,linewidth=1)
                plt.axhline(y=weighted_mean, color='tab:olive', linestyle='-', label='wmean ADC/V')
                plt.fill_between(range(-1,len(slopes)+1), (weighted_mean-weighted_std_dev), (weighted_mean+weighted_std_dev), color='tab:olive', alpha=0.2, label= '${\sigma}_{weightStd}$')
                #plt.fill_between(range(-1,len(slopes1)+1), (weighted_mean1-combined_sigma1), (weighted_mean1+combined_sigma1), color='tab:cyan', alpha=0.2, label= '${\sigma}_{weightStd} + {\sigma}_{intervar} $')
                # plt.axhline(y=np.mean(slopes1), color='tab:olive', linestyle='-', label='mean ADC/V')
                # plt.fill_between(range(-1,len(slopes1)+1), (np.mean(slopes1)-np.std(slopes1)), (np.mean(slopes1)+np.std(slopes1)), color='tab:olive', alpha=0.2)
                plt.title(f'Sensors char at {wl} nm. Value =  {weighted_mean:.1f} +- {combined_sigma:.1f} ADC/V ({100*combined_sigma/weighted_mean:.1f}%)')
                plt.xlabel('SensorID')
                plt.ylabel('ADC counts/V')
                plt.xticks(np.arange(len(slopes)), sensorsID, rotation=20)
                plt.xlim([-1,len(sensorsID)]) 
                plt.grid()
                plt.legend()
                plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_calibration.png',dpi=199)  # Display the current figure
                plt.close()
                
                if(lowLevel_plot):

                    fig = plt.figure(200)
                    plt.errorbar(range(len(slopes_stderr)), slopes_stderr, fmt='.',color='tab:purple', markersize=10, linewidth=1)
                    plt.axhline(y=np.mean(slopes_stderr), color='tab:olive', linestyle='-', label='mean ADC/V')
                    plt.fill_between(range(-1,len(slopes_stderr)+1), (np.mean(slopes_stderr)-np.std(slopes_stderr)), (np.mean(slopes_stderr)+np.std(slopes_stderr)), color='tab:olive', alpha=0.2)
                    plt.title(f'Stderr at {wl} nm. Value = {np.mean(slopes_stderr):.1f} +- {np.std(slopes_stderr):.1f}')
                    plt.xlabel('SensorID')
                    plt.ylabel('ADC counts/V')
                    plt.xticks(np.arange(len(slopes_stderr)), sensorsID, rotation=20)
                    plt.xlim([-1,len(sensorsID)])
                    plt.grid()
                    plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_calibration_stderrs_0_3.png',dpi=199)  # Display the current figure
                    plt.close()

        if(pedestal_plot):
            fig =  plt.figure(11)
            plt.errorbar(range(len(pedestalsADC)),pedestalsADC, pedestalsADC_stderr, fmt='.',color='tab:purple', markersize=10, linewidth=1)
            plt.xlabel('SensorID')
            plt.ylabel('ADC counts')
            plt.xticks(range(len(pedestalsADC)), sensorsID, rotation=20)
            plt.xlim([-1,len(sensorsID)])
            plt.grid()
            plt.title(f'Pedestal of the ADC for measurement of {wl} laser, {filter},{sensorID}')
            plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_pedestals_ADC.png',dpi=199)  # Display the current figure
            plt.close()
            
            fig =  plt.figure(12)
            plt.errorbar(range(len(pedestalsRefPD)),pedestalsRefPD, pedestalsRefPD_stderr, fmt='.',color='tab:purple', markersize=10, linewidth=1)
            plt.xlabel('SensorID')
            plt.ylabel('V')
            plt.xticks(range(len(pedestalsADC)), sensorsID, rotation=20)
            plt.xlim([-1,len(sensorsID)])
            plt.grid()
            plt.title(f'Pedestals of the PD reference for measurement of {wl} laser, {filter},{sensorID}')
            plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_pedestals_RefPD.png',dpi=199)  # Display the current figure
            plt.close()

In [19]:
def ADC_analysis_sensor_gain_combinedWithSetup1(folder_path,dataset_number,output_path,output_path_lowlevel,datetest,boardname):

    outputpath = f"{output_path}"
    outputpath_lowlevel = f"{output_path_lowlevel}"

    # We separate here the left and right side of the setup and the resistance of the sensors, so that the gain is the good one. 
    resistance = ['8K45','8K45','8K45','8K45','8K45','8K45','45K','45K','45K','45K']
    filterOrder= ['FW4', 'FW4','FW4','FW4','FW4','FW4','FW1','FW1','FW1','FW1'] 
    sensorsID = ['0.0', '0.1', '1.0', '1.1', '2.0', '2.1', '3.0', '3.1', '4.0', '4.1']
   
    #resistance = ['8K45','8K45','8K45','8K45','8K45','8K45','45K','45K','45K','45K']
    #filterOrder= ['FW4', 'FW4','FW4','FW4','FW4','FW4','FW1','FW1','FW1','FW1'] 
    #sensorsID = ['0.2', '0.3','1.2', '1.3','2.2', '2.3', '3.2','3.3', '4.2', '4.3']


    wavelength = ['1064']#,'532']

    sensor1Coefficients = [354.26, 354.26, 354.26, 354.26, 354.26, 354.26, 53.62, 53.62, 53.62, 53.62] #en microW/V
    #sensor1Coefficients = [53.62, 53.62, 53.62, 53.62] #en microW/V

    overall_W = []
    overall_ADC = []

    if not os.path.exists(outputpath_lowlevel):
        os.makedirs(outputpath_lowlevel)
        os.makedirs(outputpath)
        print(f"Directory '{outputpath_lowlevel}' created.")

    if not os.path.exists(outputpath):
        os.makedirs(outputpath)
        print(f"Directory '{outputpath}' created.")

    for wl in wavelength:
        pedestalsADC = []
        pedestalsADC_stderr = []
        pedestalsRefPD = []
        pedestalsRefPD_stderr = []
        slopes = []
        intercepts= []
        rCoes = []
        slopes_stderr = []
        intercepts_stderr = []
        saturationADCs = []
        if wl== '532':
        # Rename DataFrame columns
            strg_L = 'Laser Current (mA)'
            #temp = '20'
        elif wl == '1064':
            strg_L = 'Laser Power (mW)'
            #temp = '25'

        for l, sensorID in enumerate(sensorsID):
            filter = filterOrder[l]
            pedestal_list = []
            main_list = []
            for number in np.arange(1,dataset_number+1):
                filename = f'{folder_path}/{datetest}_{boardname}_{sensorID}_{wl}_{filter}_{number}.txt'
                # Read the first line as a separate DataFrame
                df = pd.read_csv(filename, sep="\t", header=None)
                
                # Split the first row into a separate DataFrame
                first_row = df.iloc[[0]]  # Select the first row as a DataFrame
                last_row = df.iloc[[-1]]
                pedestal_list.append(first_row)  # Add the first row to pedestal list
                pedestal_list.append(last_row)  # Add the first row to pedestal list

                # Get the remaining rows (from the second row onward)
                main_data = df.iloc[1:-1]  # Select all rows except the first and last
                main_list.append(main_data)  # Add to main data list

            # Concatenate the first rows (metadata or header info) and the main data
            TP = pd.concat(pedestal_list, ignore_index=True)
            T = pd.concat(main_list, ignore_index=True)
            TP.columns = ['Date-Hour', 'L', 'TotalSum', 'TotalSquareSum', 'meanRefPD_V', 'stdRefPD_V', 'Tem', 'RH', 'TotalCounts']
            T.columns = ['Date-Hour', 'L', 'TotalSum', 'TotalSquareSum', 'meanRefPD_V', 'stdRefPD_V', 'Tem', 'RH', 'TotalCounts']

            meanADC = T['TotalSum']/T['TotalCounts']
            stdADC = np.sqrt((T['TotalSquareSum']-T['TotalCounts']*meanADC**2)/(T['TotalCounts']-1))
            T['meanADC'] = meanADC
            T['stdADC'] = stdADC
            
            T['meanRefPD'] = T['meanRefPD_V'] * sensor1Coefficients[l] # Convert to W with the coeffiecient W/V
            T['stdRefPD'] = T['stdRefPD_V'] * sensor1Coefficients[l] # Convert to W with the coeffiecient W/V

            overall_W.append(T['meanRefPD'])
            overall_ADC.append(T['meanADC'])
            print(f'Starting the fitting procedure for calibration coefficient for sensorID {sensorID} for laser {wl} nm')

            pedestal_meanADC = TP['TotalSum']/TP['TotalCounts']
            TP['meanADC'] = pedestal_meanADC
            TP['stdADC'] = np.sqrt((TP['TotalSquareSum']-TP['TotalCounts']*pedestal_meanADC**2)/(TP['TotalCounts']-1))
            TP['meanRefPD'] = TP['meanRefPD_V'] * sensor1Coefficients[l] # Convert to W with the coeffiecient W/V
            TP['stdRefPD'] = TP['stdRefPD_V'] * sensor1Coefficients[l] # Convert to W with the coeffiecient W/V

            if fit:
                T, saturationADC = find_saturation_from_derivate(T)
                res = linregress(T['meanADC'],T['meanRefPD'])
                
                slopes.append(res.slope)
                intercepts.append(res.intercept)
                rCoes.append(res.rvalue)
                slopes_stderr.append(res.stderr)
                intercepts_stderr.append(res.intercept_stderr)
                saturationADCs.append(saturationADC)
            
            pedestalsADC_wmean, pedestalsADC_wstderr = ObtainMeanAndStd(TP['meanADC'],TP['stdADC'])
            pedestalsPD_wmean, pedestalsPD_wstderr = ObtainMeanAndStd(TP['meanRefPD'],TP['stdRefPD'])


            pedestalsADC.append(pedestalsADC_wmean)
            pedestalsADC_stderr.append(pedestalsADC_wstderr) # This errors should be added in quadrature
            pedestalsRefPD.append(pedestalsPD_wmean) 
            pedestalsRefPD_stderr.append(pedestalsPD_wstderr)# This errors should be added in quadrature


            if(verbose and fit):
                print(f'Calibration coefficient found for sensorID {sensorID} for laser {wl} nm = {res.slope} +/- {res.stderr} in ADC/V')

            if(lowLevel_plot):
                fig = plt.figure(2)
                plt.errorbar(T['L'], T['meanADC'], yerr=T['stdADC'], fmt='.', markersize=10, linewidth=1)
                plt.ylabel('mean ADC counts')
                plt.xlabel(strg_L)
                plt.grid()
                plt.title(f'Plot sensorID {sensorID}, wavelength {wl} nm, filter = {filter}')
                plt.tight_layout()
                plt.savefig(f'{outputpath_lowlevel}/{datetest}_{boardname}_{sensorID}_{wl}_meanADC_L.png',dpi=199)  # Display the current figure
                plt.close(fig)

                fig = plt.figure(4)
                plt.errorbar(T['L'], T['meanRefPD'], yerr=T['stdRefPD'], fmt='.', markersize=10, linewidth=1)
                plt.ylabel('V)')
                plt.xlabel(strg_L)
                plt.grid()
                plt.title(f'Plot sensorID {sensorID}, wavelength {wl} nm, filter = {filter}')
                plt.tight_layout()
                plt.savefig(f'{outputpath_lowlevel}/{datetest}_{boardname}_{sensorID}_{wl}_meanRefPD_L.png',dpi=199)  # Display the current figure
                #plt.show()
                plt.close(fig)


                if fit:
                    fig = plt.figure(6)
                    plt.errorbar(T['meanADC'],T['meanRefPD'], yerr=T['stdRefPD'], fmt='.', markersize=10, linewidth=1)
                    plt.plot(T['meanADC'], res.intercept + res.slope*T['meanADC'], 'r', label='fitted line')
                    plt.xlabel('Mean ADC counts')
                    plt.ylabel('Mean ref PD (V)')
                    plt.grid()
                    plt.title(f'Plot sensorID {sensorID}, wavelength {wl} nm, filter = {filter}')
                    plt.tight_layout()
                    plt.savefig(f'{outputpath_lowlevel}/{datetest}_{boardname}_{sensorID}_{wl}_meanADC_meanRefPD.png',dpi=199)  # Display the current figure
                    #plt.show()
                    plt.close(fig)
            
        if(highLevel_plot and fit):
                fig = plt.figure(3000)
                import matplotlib.cm as cm 
                #Plot the linear behaviour all together
                colors = cm.rainbow(np.linspace(0, 1, len(sensorsID)))  # generate N distinct colors
                for i, ID in enumerate(sensorsID):
                    plt.plot(overall_ADC[i],overall_W[i], marker='o', linestyle='None', color=colors[i], label=f'{ID}, {filterOrder[i]}, {resistance[i]}')
                    
                    # Add these values to a DataFrame for further analysis and to a .txt file
                    filename = f'{outputpath}/{datetest}_{boardname}_{wl}_{ID}_PowerRange.txt'
                    # Stack data into two columns: ADC and W
                    data = []
                    data = np.column_stack((overall_ADC[i], overall_W[i]))
                    # Save to file with header
                    np.savetxt(filename, data, header="ADCcounts\tW[muW]", fmt="%.6f", delimiter="\t")
                # Optional: add labels/legend
                # Lets add here the maximum power for an input power de 392KW al input tower, el power màxim segons la simulació és: R1-3: 70uW, R4-5: 42uW
                plt.axhline(y=70*190/392, color='orchid', linestyle='--', label='Max power R1-3 (190W)')
                plt.axhline(y=70, color='orchid', linestyle='--', label='Max power R1-3 (392W)')
                plt.axhline(y=42*190/392, color='yellowgreen', linestyle='--', label='Max power R4-5 (190W)')
                plt.axhline(y=42, color='yellowgreen', linestyle='--', label='Max power R4-5 (392W)')
                plt.xlabel('ADC counts')
                plt.ylabel('Power[muW]')
                plt.title('Sensor Data by ID')
                plt.legend(loc='best')  # remove this if too many sensors
                plt.yscale('log')
                plt.grid(True)
                plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_PowerRange.png',dpi=199)  # Display the current figure
                plt.close()

    

                tabdata = pd.DataFrame()
                tabdata["sensorID"] = sensorsID
                tabdata["slope"] = slopes
                tabdata["intercept"] = intercepts
                tabdata["rCoe"] = rCoes
                tabdata["slopes_stderr"] = slopes_stderr
                tabdata["intercept_stderr"] = intercepts_stderr
                tabdata["saturationADC"] = saturationADCs

                # Save these numbers in a .txt file
                selected_columns = tabdata[['sensorID', 'slope','slopes_stderr']]

                # Write to txt file (tab-separated as an example)
                selected_columns.to_csv(f'/Users/mseglar/GitHub/sensors-analysis/CalibrationCoefficient_setup2_{wl}.txt', sep='\t', index=False)
                
                fig = plt.figure(300)
                plt.errorbar(range(len(intercepts)), intercepts,intercepts_stderr, fmt='.',  markersize=10, linewidth=1)
                plt.axhline(y=np.mean(intercepts), color='tab:olive', linestyle='-')
                plt.fill_between(range(-1,len(intercepts)+1), (np.mean(intercepts)-np.std(intercepts)), (np.mean(intercepts)+np.std(intercepts)), color='tab:olive', alpha=0.2)
                plt.title(f'Intercepts at {wl} nm. Value = {np.mean(intercepts):.1f} +- {np.std(intercepts):.1f}')
                plt.xlabel('SensorID')
                plt.ylabel('$V$')
                plt.xticks(np.arange(len(intercepts)), sensorsID, rotation=20)
                plt.xlim([-1,len(sensorsID)])
                plt.grid()
                plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_noshade_intercepts.png',dpi=199)  # Display the current figure
                plt.close()
                
                fig = plt.figure(400)
                plt.errorbar(range(len(rCoes)), rCoes, fmt='.', markersize=10,linewidth=1)
                plt.axhline(y=np.mean(rCoes), color='tab:olive', linestyle='-')
                plt.fill_between(range(-1,len(rCoes)+1), (np.mean(rCoes)-np.std(rCoes)), (np.mean(rCoes)+np.std(rCoes)), color='tab:olive', alpha=0.2)
                plt.title(f'rCoes at {wl} nm. Value = {np.mean(rCoes):.1f} +- {np.std(rCoes):.1f}')
                plt.xlabel('SensorID')
                plt.ylabel('')
                plt.xticks(np.arange(len(rCoes)), sensorsID, rotation=20)
                plt.xlim([-1,len(sensorsID)])
                plt.grid()
                plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_rcoes.png',dpi=199)  # Display the current figure
                plt.close()
                
                fig = plt.figure(500)
                plt.title(f'Saturation ADC at {wl} nm. Value = {np.mean(saturationADCs):.1f} +- {np.std(saturationADCs):.1f}')
                plt.plot(range(len(saturationADCs)), saturationADCs,'.')
                plt.axhline(y=np.mean(saturationADCs), color='tab:olive', linestyle='-')
                plt.fill_between(range(-1,len(saturationADCs)+1), (np.mean(saturationADCs)-np.std(saturationADCs)), (np.mean(saturationADCs)+np.std(saturationADCs)), color='tab:olive', alpha=0.2)
                plt.xlabel('SensorID')
                plt.ylabel('ADC counts')
                plt.xticks(np.arange(len(saturationADCs)), sensorsID, rotation=20)
                plt.xlim([-1,len(sensorsID)])
                plt.grid()
                plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_saturationADC.png',dpi=199)  # Display the current figure
                plt.close()

                #Uncertainty of Weighted Mean: Precision based on individual measurement uncertainties.
                #Intervariability: Variability among the measurements (random error, i.e. embedded systematic uncertainties).
                #Purely Systematic Error: Accounts for known or estimated biases in the measurements (temperature/humidity????)

                weights = 1 / np.array(slopes_stderr)**2
                weighted_mean = np.sum(weights * slopes) / np.sum(weights)
                weighted_mean_error = np.sqrt(1 / np.sum(weights))
                weighted_variance = np.sum(weights * (slopes - weighted_mean)**2) / np.sum(weights)
                weighted_std_dev = np.sqrt(weighted_variance)
                intervariability = np.std(slopes, ddof=1)  # Standard deviation (sample)
                combined_sigma = np.sqrt(weighted_std_dev**2+intervariability**2)
                
                mean = np.mean(slopes)
                std = np.std(slopes)
               
                print('--------------------------------------------------------------------------------------------')
                print(f'Mean characterization sensors, {wl} = {mean} +- {std}. The error in percent is: {100*std/mean}$%$')
                print(f'Weighted mean sensors, {wl} = {weighted_mean} +- {weighted_mean_error}. The error in percent is: {100*weighted_mean_error/weighted_mean}$%$')
                print(f'Weighted standard deviation (spread): {weighted_std_dev}')
                print(f'Standard deviation of the measurements, intervariability: {intervariability}')
                print(f'Total combined error:{combined_sigma}')
                print(f'=> Weighted characterization sensor, {wl} = {weighted_mean} +- {combined_sigma}. The error in percent is: {100*combined_sigma/weighted_mean}$%$')

                
                print('-------------------------------------------------------------------------------------------')


                fig = plt.figure(101)
                plt.errorbar(range(len(slopes)),slopes,slopes_stderr,fmt='.',color='tab:purple',markersize=10,linewidth=1)
                plt.axhline(y=weighted_mean, color='tab:olive', linestyle='-', label='wmean V/ADC')
                plt.fill_between(range(-1,len(slopes)+1), (weighted_mean-weighted_std_dev), (weighted_mean+weighted_std_dev), color='tab:olive', alpha=0.2, label= '${\sigma}_{weightStd}$')
                #plt.fill_between(range(-1,len(slopes1)+1), (weighted_mean1-combined_sigma1), (weighted_mean1+combined_sigma1), color='tab:cyan', alpha=0.2, label= '${\sigma}_{weightStd} + {\sigma}_{intervar} $')
                # plt.axhline(y=np.mean(slopes1), color='tab:olive', linestyle='-', label='mean ADC/V')
                # plt.fill_between(range(-1,len(slopes1)+1), (np.mean(slopes1)-np.std(slopes1)), (np.mean(slopes1)+np.std(slopes1)), color='tab:olive', alpha=0.2)
                plt.title(f'Sensors char at {wl} nm. Value =  {weighted_mean:.1f} +- {combined_sigma:.1f} V/ADC ({100*combined_sigma/weighted_mean:.1f}%)')
                plt.xlabel('SensorID')
                plt.ylabel('$V/ADC counts')
                plt.xticks(np.arange(len(slopes)), sensorsID, rotation=20)
                plt.xlim([-1,len(sensorsID)]) 
                plt.grid()
                plt.legend()
                plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_calibration.png',dpi=199)  # Display the current figure
                plt.close()
                
                if(lowLevel_plot):

                    fig = plt.figure(200)
                    plt.errorbar(range(len(slopes_stderr)), slopes_stderr, fmt='.',color='tab:purple', markersize=10, linewidth=1)
                    plt.axhline(y=np.mean(slopes_stderr), color='tab:olive', linestyle='-', label='V/mean ADC')
                    plt.fill_between(range(-1,len(slopes_stderr)+1), (np.mean(slopes_stderr)-np.std(slopes_stderr)), (np.mean(slopes_stderr)+np.std(slopes_stderr)), color='tab:olive', alpha=0.2)
                    plt.title(f'Stderr at {wl} nm. Value = {np.mean(slopes_stderr):.1f} +- {np.std(slopes_stderr):.1f}')
                    plt.xlabel('SensorID')
                    plt.ylabel('V/ADC counts')
                    plt.xticks(np.arange(len(slopes_stderr)), sensorsID, rotation=20)
                    plt.xlim([-1,len(sensorsID)])
                    plt.grid()
                    plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_calibration_stderrs_0_3.png',dpi=199)  # Display the current figure
                    plt.close()

        if(pedestal_plot):
            fig =  plt.figure(11)
            plt.errorbar(range(len(pedestalsADC)),pedestalsADC, pedestalsADC_stderr, fmt='.',color='tab:purple', markersize=10, linewidth=1)
            plt.xlabel('SensorID')
            plt.ylabel('ADC counts')
            plt.xticks(range(len(pedestalsADC)), sensorsID, rotation=20)
            plt.xlim([-1,len(sensorsID)])
            plt.grid()
            plt.title(f'Pedestal of the ADC for measurement of {wl} laser, {filter},{sensorID}')
            plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_pedestals_ADC.png',dpi=199)  # Display the current figure
            plt.close()
            
            fig =  plt.figure(12)
            plt.errorbar(range(len(pedestalsRefPD)),pedestalsRefPD, pedestalsRefPD_stderr, fmt='.',color='tab:purple', markersize=10, linewidth=1)
            plt.xlabel('SensorID')
            plt.ylabel('V')
            plt.xticks(range(len(pedestalsADC)), sensorsID, rotation=20)
            plt.xlim([-1,len(sensorsID)])
            plt.grid()
            plt.title(f'Pedestals of the PD reference for measurement of {wl} laser, {filter},{sensorID}, in W')
            plt.savefig(f'{outputpath}/{datetest}_{boardname}_{wl}_pedestals_RefPD.png',dpi=199)  # Display the current figure
            plt.close()

## ANALYSIS OF THE DATASET

In [20]:
#ADC_analysis_sensor_gain('./Data/setup2/25112025_19L2/24112025_19L2/data',3,"./Data/setup2/25112025_19L2/24112025_19L2/plots","./Data/setup2/25112025_19L2/24112025_19L2/plots/LowLevel",'24112025','19L2')
#ADC_analysis_sensor_gain('./Data/setup2/25112025_19L2/20112025_19L2/data',3,"./Data/setup2/25112025_19L2/20112025_19L2/plots","./Data/setup2/25112025_19L2/20112025_19L2/plots/LowLevel",'20112025','19L2')
# ADC_analysis_sensor_gain('./Data/setup2/05112025_16R0/31102025_16R0/data',3,"./Data/setup2/05112025_16R0/31102025_16R0/plots","./Data/setup2/05112025_16R0/31102025_16R0/plots/LowLevel",'31102025','16R0')
ADC_analysis_sensor_gain('./Data/setup2/01122025_28R0/data',3,"./Data/setup2/01122025_28R0/plots","./Data/setup2/01122025_28R0/plots/LowLevel",'01122025','28R0')


Starting the fitting procedure for calibration coefficient for sensorID 0.2 for laser 532 nm
Calibration coefficient found for sensorID 0.2 for laser 532 nm = 9608.102854595405 +/- 376.7732907457099 in ADC/V


/opt/miniconda3/envs/readPandas/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/miniconda3/envs/readPandas/lib/python3.11/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


--------------------------------------------------------------------------------------------
Mean characterization sensors, 532 = 9608.102854595405 +- 0.0. The error in percent is: 0.0$%$
Weighted mean sensors, 532 = 9608.102854595405 +- 376.7732907457099. The error in percent is: 3.921411921246296$%$
Weighted standard deviation (spread): 0.0
Standard deviation of the measurements, intervariability: nan
Total combined error:nan
=> Weighted characterization sensor, 532 = 9608.102854595405 +- nan. The error in percent is: nan$%$
-------------------------------------------------------------------------------------------


## Stability

In [ ]:
def MergeDatasets(folder_path):
    # Initialize an empty list to store individual DataFrames
    dataframes = []
    # Iterate over each file in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):  # Only process .txt files
            file_path = os.path.join(folder_path, file_name)
            sensor_id = file_name.split('.')[0]

            # Read the file into a pandas DataFrame
            df = pd.read_csv(file_path, sep='\t', header=None)
            
            # Assign the provided column names
            df.columns = ['Date-Time', 'Laser_Power', 'Total_Sum', 'Total_Sqr', 'V_Mean', 'Std_V', 'Temp', 'Humidity', 'Total_Counts']
            df.loc[:, 'pd_val'] = df['Total_Sum']/df['Total_Counts']
            df.loc[:, 'pd_val_norm'] = df['pd_val']/df['V_Mean']
            df.loc[:, 'sensor_id'] = sensor_id
            
            # Append the DataFrame to the list
            dataframes.append(df)

    # Concatenate all the DataFrames into one
    final_df = pd.concat(dataframes, ignore_index=True)

    # Display the first few rows of the concatenated DataFrame
    #print(final_df.head())
    return final_df

In [ ]:
def Overall_Stability(final_df, columns_to_plot):
    # Create a plot for each column
    for column in columns_to_plot:
        # Calculate mean and variance
        mean_value = final_df[column].mean()
        std_dev = final_df[column].std()  # Standard deviation
        
        # Calculate sigma as a percentage of the mean
        sigma_percentage = (std_dev / mean_value) * 100 if mean_value != 0 else 0
        
        # Print mean and sigma (standard deviation)
        print(f'{column}:')
        print(f'  Mean: {mean_value}')
        print(f'  Standard Deviation (σ): {std_dev}')
        print(f'  Standard Deviation as Percentage of Mean: {sigma_percentage}%')
        
        
        # Create a subplot with 2 plots side by side (1 row, 2 columns)
        fig, axs = plt.subplots(1, 2, figsize=(15, 6))  # Adjust the figure size as necessary

        # Plot the evolution of values (line plot) on the left
        axs[0].plot(final_df[column], '.', color='orange')
        axs[0].set_title(f'{column} Evolution (Mean: {mean_value:.2f}, σ%: {sigma_percentage:.2f}%)')
        axs[0].set_xlabel('Index (Time Steps)')
        axs[0].set_ylabel(column)
        axs[0].grid(True)

        # Plot the histogram on the right
        axs[1].hist(final_df[column], bins=30, edgecolor='black', alpha=0.7)
        axs[1].set_title(f'{column} Histogram')
        axs[1].set_xlabel(column)
        axs[1].set_ylabel('Frequency')
        axs[1].grid(True)

        # Adjust the layout to prevent overlapping
        plt.tight_layout()
        
        # Show the combined plots
        plt.show()

## Check overall stability of the main variables

In [ ]:
datasets = MergeDatasets('./Data/setup2/28102024_4L2/data')
# Plot each column except 'Date' and 'Time'
columns_to_plot = ['Laser_Power', 'Total_Sum', 'Total_Sqr', 'V_Mean', 'Std_V', 'Temp', 'Humidity', 'Total_Counts']

Overall_Stability(datasets,columns_to_plot)

In [ ]:
datasets = MergeDatasets('./Data/setup2/20112024_9R1/data')
# Plot each column except 'Date' and 'Time'
columns_to_plot = ['Laser_Power', 'Total_Sum', 'Total_Sqr', 'V_Mean', 'Std_V', 'Temp', 'Humidity', 'Total_Counts']

Overall_Stability(datasets,columns_to_plot)